In [39]:
%use kandy
%use lib-ext

In [40]:
@file:DependsOn("com.ustermetrics:clarabel4j:1.0.0")
@file:DependsOn("com.ustermetrics:clarabel4j-native:1.0.0-0.9.0")
@file:DependsOn("org.ejml:ejml-all:0.43.1")

In [41]:
import com.ustermetrics.clarabel4j.Model
import com.ustermetrics.clarabel4j.Parameters
import com.ustermetrics.clarabel4j.Status.SOLVED
import com.ustermetrics.clarabel4j.Matrix
import com.ustermetrics.clarabel4j.NonnegativeCone
import com.ustermetrics.clarabel4j.SecondOrderCone
import com.ustermetrics.clarabel4j.ZeroCone
import org.ejml.data.DMatrixSparseCSC
import org.ejml.dense.row.factory.DecompositionFactory_DDRM
import org.ejml.ops.DConvertMatrixStruct
import org.ejml.simple.SimpleMatrix

# Portfolio Optimization
        
A long-only investor wishes to maximize the expected portfolio return given a limit on the portfolio risk

$$
\begin{align*}
& \text{maximize} & & \mu^T x \\
& \text{subject to} & & x^T \Sigma x \leq \sigma^2 \\
& & & \mathbf{1} x = 1 \\
& & & x \geq 0
\end{align*}
$$

where $x$ is the unknown vector of portfolio allocations, $\mu$ is the estimated expected return vector, $\Sigma$ is the
estimated covariance matrix, and $\sigma$ is the given limit on the portfolio risk.

In [42]:
// Define portfolio optimization problems
val mu = SimpleMatrix(doubleArrayOf(0.05, 0.09, 0.07, 0.06))
val sigma = SimpleMatrix(
    4, 4, true,
    0.0016, 0.0006, 0.0008, -0.0004,
    0.0006, 0.0225, 0.0015, -0.0015,
    0.0008, 0.0015, 0.0025, -0.001,
    -0.0004, -0.0015, -0.001, 0.01
)
val sigmaLimits = List(20) { 0.0321 + it * (0.06 - 0.0321) / 19 }

In [43]:
// Problem dimension
val n = mu.numRows + 1

In [44]:
// Compute Cholesky decomposition of sigma
val chol = DecompositionFactory_DDRM.chol(n - 1, true)
if (!chol.decompose(sigma.copy().getMatrix()))
    throw IllegalArgumentException("Cholesky decomposition failed")
val upTriMat = SimpleMatrix.wrap(chol.getT(null)).transpose()

// Define second-order cone program
val qMat = mu.negative()
    .concatRows(SimpleMatrix(1, 1))
println("\nqMat")
qMat.print()

val aMatZeroCone = SimpleMatrix.ones(1, n - 1)
    .concatColumns(SimpleMatrix(1, 1))
val aMatNonNeg = SimpleMatrix.identity(n - 1)
    .negative()
    .concatColumns(SimpleMatrix(n - 1, 1))
    .concatRows(SimpleMatrix(1, n - 1).concatColumns(SimpleMatrix.ones(1, 1)))
val aMatSoc = SimpleMatrix(1, n - 1)
    .concatColumns(SimpleMatrix.filled(1, 1, -1.0))
    .concatRows(upTriMat.negative().concatColumns(SimpleMatrix(n - 1, 1)))
val aMat = aMatZeroCone.concatRows(aMatNonNeg).concatRows(aMatSoc)
println("\naMat")
aMat.print()

val bMat = SimpleMatrix(2 * (n - 1) + 3, 1)
bMat.set(0, 0, 1.0)


qMat
Type = DDRM , rows = 5 , cols = 1
-.05       
-.09       
-.07       
-.06       
 0         

aMat
Type = DDRM , rows = 11 , cols = 5
 1           1           1           1           0         
-1          -0          -0          -0           0         
-0          -1          -0          -0           0         
-0          -0          -1          -0           0         
-0          -0          -0          -1           0         
 0           0           0           0           1         
 0           0           0           0          -1         
-.04        -.015       -.02         .01         0         
-0          -.149248116 -.008040303  .00904534   0         
-0          -0          -.045114893  .016120458  0         
-0          -0          -0          -.097766623  0         


In [45]:
// ecos4j needs sparse aMat and gMat
val tol = 1e-8
val aSpMat = DConvertMatrixStruct.convert(aMat.ddrm, null as DMatrixSparseCSC?, tol)
println("\naSpMat")
aSpMat.print()


aSpMat
Type = DSCC , rows = 11 , cols = 5 , nz_length = 20
 1           1           1           1               *     
-1               *           *           *           *     
     *      -1               *           *           *     
     *           *      -1               *           *     
     *           *           *      -1               *     
     *           *           *           *       1         
     *           *           *           *      -1         
-.04        -.015       -.02         .01             *     
     *      -.149248116 -.008040303  .00904534       *     
     *           *      -.045114893  .016120458      *     
     *           *           *      -.097766623      *     


In [46]:
// Helper functions
fun convert(matrix: DMatrixSparseCSC): Matrix {
    return Matrix(matrix.getNumRows(), matrix.getNumCols(), getColIdx(matrix), getNzRows(matrix), getNzValues(matrix))
}

fun getNzValues(matrix: DMatrixSparseCSC): DoubleArray {
    if (!matrix.isIndicesSorted()) matrix.sortIndices(null);
    if (matrix.nz_values.size == matrix.nz_length) {
        return matrix.nz_values;
    } else {
        return matrix.nz_values.copyOfRange(0, matrix.nz_length);
    }
}

fun getNzRows(matrix: DMatrixSparseCSC): LongArray {
    if (!matrix.isIndicesSorted()) matrix.sortIndices(null);
    if (matrix.nz_rows.size == matrix.nz_length) {
        return toLongArray(matrix.nz_rows);
    } else {
        return toLongArray(matrix.nz_rows.copyOfRange(0, matrix.nz_length));
    }
}

fun getColIdx(matrix: DMatrixSparseCSC): LongArray {
    return toLongArray(matrix.col_idx);
}

fun toLongArray(arr: IntArray): LongArray {
    return arr.map { it.toLong() }.toLongArray()
}

In [47]:
val muPortfolio = mutableListOf<Double>()
val sigmaPortfolio = mutableListOf<Double>()
lateinit var xMat: SimpleMatrix

// Compute efficient frontier
sigmaLimits.forEach { sigmaLimit ->

    // Set sigma limit
    bMat.set(n, 0, sigmaLimit)

    Model().use { model ->

        // Create and set parameters
        val parameters = Parameters.builder()
            .verbose(true)
            .build()
        model.setParameters(parameters)

        // Set up model
        model.setup(
            qMat.getDDRM().data, convert(aSpMat), bMat.ddrm.data,
            listOf(ZeroCone(1), NonnegativeCone(n.toLong()), SecondOrderCone(n.toLong()))
        )

        // Optimize model
        val status = model.optimize()
        if (status != SOLVED)
            throw IllegalStateException("Optimization failed")

        // Get solution
        xMat = SimpleMatrix(model.x())
            .extractMatrix(0, n - 1, 0, 1)

        // Compute portfolio return and risk
        muPortfolio.add(mu.transpose().mult(xMat).get(0, 0))
        sigmaPortfolio.add(sqrt(xMat.transpose().mult(sigma).mult(xMat).get(0, 0)))
    }
}

In [48]:
// Plot efficient frontier
plot {
    layout.title = "Efficient frontier"
    smoothLine(sigmaPortfolio, muPortfolio, method = SmoothMethod.LOESS(span = 0.1)) {
        color = Color.GREEN
    }
    points {
        x(sigmaPortfolio) {
            axis {
                name = "Risk"
            }
        }
        y(muPortfolio) {
            axis {
                name = "Return"
            }
        }
        size = 3.0
        color = Color.GREEN
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="r6dalM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Efficient frontier"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"name":"Risk",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Return",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[0.03210000028482681,0.03238181845657443,0.03266363662832205,0.03294545480006967,0.03322727297181729,0.03350909114356491,0.03379090931531253,0.034072727487060146,0.034354545658807766,0.034636363830555385,0.034918182002303004,0.03520000017405062,0.03548181834579824,0.03576363651754586,0.03604545468929348,0.0363272728610411,0.03660909103278872,0.03689090920453634,0.03717272737628396,0.03745454554803158,0.037736363719779197,0.038018181891526816,0.038300000063274435,0.038581818235022054,0.038863636406769674,0.03914545457851729,0.03942727275026491,0.03970909092201253,0.03999090909376015,0.04027272726550777,0.04055454543725539,0.04083636360900301,0.04111818178075063,0.04139999995249825,0.041681818124245866,0.041963636295993485,0.042245454467741105,0.042527272639488724,0.04280909081123634,0.04309090898298396,0.04337272715473158,0.0436545453264792,0.04393636349822682,0.04421818166997444,0.04449999984172206,0.04478181801346968,0.0450636361852173,0.045345454356964916,0.045627272528712536,0.045909090700460155,0.046190908872207774,0.04647272704395539,0.04675454521570301,0.04703636338745063,0.04731818155919825,0.04759999973094587,0.04788181790269349,0.04816363607444111,0.04844545424618873,0.04872727241793635,0.04900909058968397,0.049290908761431586,0.049572726933179205,0.049854545104926824,0.050136363276674444,0.05041818144842206,0.05069999962016968,0.0509818177919173,0.05126363596366492,0.05154545413541254,0.05182727230716016,0.05210909047890778,0.0523909086506554,0.05267272682240302,0.052954544994150636,0.053236363165898255,0.053518181337645875,0.053799999509393494,0.05408181768114111,0.05436363585288873,0.05464545402463635,0.05492727219638397,0.05520909036813159,0.05549090853987921,0.05577272671162683,0.05605454488337445,0.05633636305512207,0.056618181226869686,0.056899999398617306,0.057181817570364925,0.057463635742112544,0.05774545391386016,0.05802727208560778,0.0583090902573554,0.05859090842910302,0.05887272660085064,0.05915454477259826,0.05943636294434588,0.0597181811160935,0.05999999928784112],
"y":[0.0585341687646382,0.05944048792941719,0.060321946325136176,0.06115368318273516,0.061910837733154145,0.06256854920733312,0.0631062585836762,0.06353629657898535,0.06388617572011308,0.06418349011015007,0.06445583385218694,0.064729735127151,0.06501436308458301,0.06530476406422725,0.06559557437726257,0.06588143033486778,0.0661570306907165,0.06642002410068001,0.06667222428882343,0.06691575955723568,0.06715275820800569,0.06738534715386162,0.06761511093701851,0.06784227091551655,0.06806683767345081,0.0682888217949164,0.06850823386400838,0.06872500188887778,0.06893866697972764,0.06914865703038164,0.06935439992050424,0.0695553235297599,0.069750808249994,0.0699398024918574,0.07012102407554652,0.07029318865789234,0.07045501189572591,0.07060528585787183,0.07074436057994082,0.07087404068017131,0.0709961864814001,0.07111265830646403,0.0712253067122246,0.07133531486592601,0.071442781724648,0.07154770691209628,0.07165009005197648,0.07174993081030108,0.07184727766015385,0.07194232188412122,0.07203528074703727,0.07212637151373612,0.07221581144905186,0.07230379888685429,0.07239042810798477,0.07247575783163528

In [49]:
// Plot asset allocation with 6% risk
val assets = MutableList(n - 1) { i -> "Asset ${i + 1}" }
val allocations = MutableList(n - 1) { i -> 100 * xMat.get(i, 0) }

plot {
    layout.title = "Asset allocation with 6% risk"
    bars {
        x(assets) { axis.name = "" }
        y(allocations) { axis.name = "Allocation (%)" }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="FnqCuO"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Asset allocation with 6% risk"
},
"mapping":{
},
"data":{
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true,
"name":""
},{
"aesthetic":"y",
"name":"Allocation (%)",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Asset 1","Asset 2","Asset 3","Asset 4"],
"y":[1.1739378732492972E-6,29.676395185620848,66.34060318120689,3.983000459234394]
},
"sampling":"none",
"position":"dodge",
"geom":"bar"
}]
};
 var plotContainer = document.getElementById("FnqCuO");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Asset 1 
 
 
 
 
 
 
 
 
 Asset 2 
 
 
 
 
 
 
 
 
 Asset 3 
 
 
 
 
 
 
 
 
 Asset 4 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 50 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 
 
 Asset allocation with 6% risk 
 
 
 
 
 Allocation (%)